In [103]:
from lxml import etree, html
import pandas as pd
from datetime import date
import requests

In [6]:
doc = etree.parse('BOE-A-2020-14372.xml')

In [7]:
#etree.tostring(doc,pretty_print=True ,xml_declaration=True, encoding="utf-8")

In [8]:
raiz=doc.getroot()
raiz.tag

'documento'

In [9]:
len(raiz)

4

In [10]:
libro=raiz[0]
libro.tag

'metadatos'

In [11]:
libro=raiz[0]
libro[0].text

'BOE-A-2020-14372'

In [12]:
libro=raiz[1]
libro[0].tag

'{http://www.w3.org/1999/02/22-rdf-syntax-ns#}RDF'

In [13]:
libro=raiz[1]
libro[0].text

'\n            '

In [14]:
Identificador=raiz.find("metadatos/identificador")
Titulo=raiz.find("metadatos/titulo")
URL_pdf=raiz.find("metadatos/url_pdf")

In [15]:
print(Identificador.text)
print(Titulo.text)
print("https://www.boe.es" + URL_pdf.text)


BOE-A-2020-14372
Decreto-ley 35/2020, de 20 de octubre, de modificación del Decreto-ley 19/2020, de 19 de mayo, y del Decreto-ley 29/2020, de 28 de julio, en materia de adopción de medidas sociales y sanitarias para paliar los efectos de la pandemia generada por la COVID-19.
https://www.boe.es/boe/dias/2020/11/18/pdfs/BOE-A-2020-14372.pdf


In [16]:
materia=raiz.find("analisis/materias/materia")
materia.text

'Asistencia sanitaria'

In [17]:
materias=raiz.findall("analisis/materias/materia")
for materia in materias:
    print(materia.text)

Asistencia sanitaria
Asistencia social
Ayudas
Cataluña
Centros residenciales para personas con discapacidad
Epidemias
Establecimientos residenciales para la Tercera Edad
Pagos
Servicios Sociales de las Comunidades Autónomas


In [18]:
type(materias)

list

In [19]:
alertas=raiz.findall("analisis/alertas/alerta")
for alerta in alertas:
    print(alerta.text)

Asuntos sociales
Sanidad


# Recoge resumen diario del BOE de hoy

In [20]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)



Fecha de Hoy = 20201217
dia = 17
mes = 12
año = 2020


In [21]:
URL_XML_resumen =  "https://www.boe.es/diario_boe/xml.php?id=BOE-S-" + str(hoy)

In [22]:
URL_XML_resumen

'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20201217'

In [23]:
#pip install requests for installation



url = URL_XML_resumen
r = requests.get(url)
#r.content
#r.text

In [24]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)
        
#save_html(r.content, './BOEs/resumen.xml')
save_html(r.content, './BOEs/Resumen-BOE-' + hoy + '.xml')

In [25]:
#r.encoding

In [26]:
#resumen = etree.parse('./BOEs/resumen.xml')
resumen = etree.parse('./BOEs/Resumen-BOE-' + hoy + '.xml')

In [27]:
raiz=resumen.getroot()
raiz.tag
raiz_sumario = raiz

In [28]:
seccion = raiz.findall("sumario/diario/seccion")
seccion

[]

In [29]:
for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')
    print(nombre_seccion)

['I. Disposiciones generales']
['II. Autoridades y personal. - A. Nombramientos, situaciones e incidencias']
['II. Autoridades y personal. - B. Oposiciones y concursos']
['III. Otras disposiciones']
['IV. Administración de Justicia']
['V. Anuncios. - A. Contratación del Sector Público']
['V. Anuncios. - B. Otros anuncios oficiales']
['V. Anuncios. - C. Anuncios particulares']


In [30]:
secciones = raiz.findall("sumario/diario/seccion")
for seccion in secciones:
    print(seccion.text)

In [31]:
tabla_resumen = pd.DataFrame()

In [32]:
#for seccion in raiz.xpath('//seccion[contains(@nombre, "III. Otras disposiciones") or contains(@nombre, "I. Disposiciones generales")]'):
for seccion in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales")]'):
#for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')

    for departamento in seccion:
        nombre_departamento = departamento.xpath('@nombre') 

        for epigrafe in departamento:
            nombre_epigrafe = epigrafe.xpath('@nombre')

            for item in epigrafe:
                item_id = item.xpath('@id')
                item_name = item.xpath('.//titulo/text()') 

                item_urlXml = "https://www.boe.es" + str(item.xpath('.//urlXml/text()'))[2:-2]
                #print(item_urlXml)

                tabla_resumen = tabla_resumen.append({'Seccion': nombre_seccion, 
                                                      'Departamento': nombre_departamento, 
                                                      'Epigrafe' : nombre_epigrafe,
                                                      'Item_id': item_id, 
                                                      'Item_Nombre' : item_name, 
                                                      'Item_URL_XML' : item_urlXml},
                                                      ignore_index=True)


In [33]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[JEFATURA DEL ESTADO],[Código Penal],"[Ley Orgánica 2/2020, de 16 de diciembre, de m...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16345],[I. Disposiciones generales]
1,[JEFATURA DEL ESTADO],[Agricultura. Alimentanción],"[Ley 8/2020, de 16 de diciembre, por la que se...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16346],[I. Disposiciones generales]
2,[JEFATURA DEL ESTADO],[Contaminación atmosférica],"[Ley 9/2020, de 16 de diciembre, por la que se...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16347],[I. Disposiciones generales]
3,"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],[Acuerdo Especial Multilateral RID 8.2020 en v...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16348],[I. Disposiciones generales]
4,[MINISTERIO DEL INTERIOR],[Protección civil],"[Resolución de 16 de diciembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16349],[I. Disposiciones generales]
5,"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Energía eléctrica],"[Real Decreto 1106/2020, de 15 de diciembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16350],[I. Disposiciones generales]
6,"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Subvenciones],"[Real Decreto 1107/2020, de 15 de diciembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16351],[I. Disposiciones generales]
7,[MINISTERIO DE POLÍTICA TERRITORIAL Y FUNCIÓN ...,[Organización],"[Real Decreto 1055/2020, de 1 de diciembre, po...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16352],[I. Disposiciones generales]
8,[MINISTERIO DE DERECHOS SOCIALES Y AGENDA 2030],[Subvenciones],"[Real Decreto 1112/2020, de 15 de diciembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16353],[I. Disposiciones generales]


# Descarga ficheros XML asociados

In [34]:
for item_URL in tabla_resumen['Item_URL_XML']:
    #print(URL)
    r = requests.get(item_URL)
    f = './BOEs/' + item_URL[-16:] + '.xml'
    save_html(r.content, f)

In [35]:

import glob
BOEs = glob.glob('./BOEs/BOE*.xml')
print (BOEs)

['./BOEs/BOE-A-2020-15394.xml', './BOEs/BOE-A-2020-14465.xml', './BOEs/BOE-A-2020-14459.xml', './BOEs/BOE-A-2020-15237.xml', './BOEs/BOE-A-2020-15223.xml', './BOEs/BOE-A-2020-15222.xml', './BOEs/BOE-A-2020-15236.xml', './BOEs/BOE-A-2020-14458.xml', './BOEs/BOE-A-2020-14464.xml', './BOEs/BOE-A-2020-15395.xml', './BOEs/BOE-A-2020-15397.xml', './BOEs/BOE-A-2020-14466.xml', './BOEs/BOE-A-2020-15208.xml', './BOEs/BOE-A-2020-14880.xml', './BOEs/BOE-A-2020-15220.xml', './BOEs/BOE-A-2020-15234.xml', './BOEs/BOE-A-2020-15235.xml', './BOEs/BOE-A-2020-15221.xml', './BOEs/BOE-A-2020-15209.xml', './BOEs/BOE-A-2020-14467.xml', './BOEs/BOE-A-2020-15396.xml', './BOEs/BOE-A-2020-15392.xml', './BOEs/BOE-A-2020-14463.xml', './BOEs/BOE-A-2020-15225.xml', './BOEs/BOE-A-2020-15231.xml', './BOEs/BOE-A-2020-15219.xml', './BOEs/BOE-A-2020-15218.xml', './BOEs/BOE-A-2020-15230.xml', './BOEs/BOE-A-2020-15224.xml', './BOEs/BOE-A-2020-14462.xml', './BOEs/BOE-A-2020-15393.xml', './BOEs/BOE-A-2020-15391.xml', './BOEs

In [36]:
#import os
#path = './BOEs'
#BOEs = [f for f in os.listdir(path) if f.startswith('BOE')]
#BOEs

In [37]:
tabla_materias = pd.DataFrame()

for BOE in BOEs:
    #print (BOE)
    BOE_XML = etree.parse(BOE)
    raiz=BOE_XML.getroot()

    materias = [materia.text for materia in raiz.findall('analisis/materias/materia')]
    alertas = [alerta.text for alerta in raiz.findall('analisis/alertas/alerta')]
    #Item_Id = '[' + BOE[7:23] + ']'
    #Item_Id = raiz.find('metadatos/identificador')
    Item_Id = raiz.xpath(".//identificador/text()")
 
    #print(Item_Id, materias, alertas)
    #tabla_materias.loc[['Item_id'], ['Materias']] = materias
    tabla_materias = tabla_materias.append({'Item_id': Item_Id,
                                            'Materias' : materias,
                                            'Alertas' : alertas},
                                            ignore_index=True)

    #tabla_materias.Item_id.apply(list)
    #fila_tabla = tabla_resumen.set_index('Item_id').filter(like=BOE[18:23], axis=0).reset_index()
    #print(fila_tabla['Item_id'])

    #tabla_resumen.loc[fila_tabla['Item_id']]['Materias'] = materias


In [38]:
tabla_materias.sort_values('Item_id')

,Alertas,Item_id,Materias
145,"[Relaciones internacionales, Sanidad, Sistema ...",[BOE-A-2020-14457],"[Acuerdos internacionales, Créditos Presupuest..."
7,"[Educación y enseñanza, Relaciones internacion...",[BOE-A-2020-14458],"[Acuerdos internacionales, Barcelona, Centros ..."
2,"[Educación y enseñanza, Relaciones internacion...",[BOE-A-2020-14459],"[Acuerdos internacionales, Barcelona, Centros ..."
32,"[Administración de Justicia, Sistema financiero]",[BOE-A-2020-14460],"[Ayuntamientos, Créditos Presupuestarios, Juzg..."
39,[Sistema financiero],[BOE-A-2020-14461],[Administración y Contabilidad de la Hacienda ...
...,...,...,...
82,"[Organización de la Administración, Seguridad ...",[BOE-A-2020-16349],"[Catástrofes, Cruz Roja Española, Dirección Ge..."
84,"[Consumidores y usuarios, Derecho Mercantil, E...",[BOE-A-2020-16350],"[Actividades económicas, Ayudas, Certificacion..."
94,[Asuntos sociales],[BOE-A-2020-16351],"[Ayudas, Ayuntamientos, Guerra Civil, Inhumaci..."
106,"[Administración de Justicia, Relaciones intern...",[BOE-A-2020-16352],"[Administración de Justicia, Bélgica, Cooperac..."


In [39]:
print(raiz.tag)

documento


In [40]:
for materias in raiz.xpath('.//materias'):
    materia = seccion.xpath('.//materia/text()')
    print(materia)

[]


In [41]:
materias=raiz.findall("analisis/materias/materia")
for materia in materias:
    print(materia.text)

Asociaciones
Fundaciones
Medio ambiente
Ministerio para la Transición Ecológica y el Reto Demográfico
Subvenciones


In [42]:
alertas=raiz.findall("analisis/alertas/alerta")
for alerta in alertas:
    print(alerta.text)

Medio ambiente


In [43]:
#tabla_resumen.loc[tabla_resumen['Item_id'] == '[' + BOE[7:23] + ']']
print(tabla_resumen.loc[tabla_resumen['Item_id'] == '[BOE-A-2020-14457]'])

Empty DataFrame
Columns: [Departamento, Epigrafe, Item_Nombre, Item_URL_XML, Item_id, Seccion]
Index: []


In [44]:
BOE[7:23]

'BOE-A-2020-15399'

In [45]:
tabla_resumen['Item_id']

0    [BOE-A-2020-16345]
1    [BOE-A-2020-16346]
2    [BOE-A-2020-16347]
3    [BOE-A-2020-16348]
4    [BOE-A-2020-16349]
5    [BOE-A-2020-16350]
6    [BOE-A-2020-16351]
7    [BOE-A-2020-16352]
8    [BOE-A-2020-16353]
Name: Item_id, dtype: object

In [46]:
tabla_materias['Item_id']

0      [BOE-A-2020-15394]
1      [BOE-A-2020-14465]
2      [BOE-A-2020-14459]
3      [BOE-A-2020-15237]
4      [BOE-A-2020-15223]
              ...        
150    [BOE-A-2020-15207]
151    [BOE-A-2020-15206]
152    [BOE-A-2020-15212]
153    [BOE-A-2020-15400]
154    [BOE-A-2020-15399]
Name: Item_id, Length: 155, dtype: object

In [47]:
tabla_resumen.dtypes

Departamento    object
Epigrafe        object
Item_Nombre     object
Item_URL_XML    object
Item_id         object
Seccion         object
dtype: object

In [48]:
tabla_resumen.set_index('Item_id').filter(like='14875', axis=0)

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Seccion
Item_id,,,,,


In [49]:
#tabla_resumen = tabla_resumen.astype(str)
#tabla_resumen['Joined'] = tabla_resumen.Item_id.apply(', '.join)
#tabla_resumen['Item_id']=tabla_resumen['Item_id']

In [50]:
tabla_resumen.dtypes

Departamento    object
Epigrafe        object
Item_Nombre     object
Item_URL_XML    object
Item_id         object
Seccion         object
dtype: object

In [51]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[JEFATURA DEL ESTADO],[Código Penal],"[Ley Orgánica 2/2020, de 16 de diciembre, de m...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16345],[I. Disposiciones generales]
1,[JEFATURA DEL ESTADO],[Agricultura. Alimentanción],"[Ley 8/2020, de 16 de diciembre, por la que se...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16346],[I. Disposiciones generales]
2,[JEFATURA DEL ESTADO],[Contaminación atmosférica],"[Ley 9/2020, de 16 de diciembre, por la que se...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16347],[I. Disposiciones generales]
3,"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],[Acuerdo Especial Multilateral RID 8.2020 en v...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16348],[I. Disposiciones generales]
4,[MINISTERIO DEL INTERIOR],[Protección civil],"[Resolución de 16 de diciembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16349],[I. Disposiciones generales]
5,"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Energía eléctrica],"[Real Decreto 1106/2020, de 15 de diciembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16350],[I. Disposiciones generales]
6,"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Subvenciones],"[Real Decreto 1107/2020, de 15 de diciembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16351],[I. Disposiciones generales]
7,[MINISTERIO DE POLÍTICA TERRITORIAL Y FUNCIÓN ...,[Organización],"[Real Decreto 1055/2020, de 1 de diciembre, po...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16352],[I. Disposiciones generales]
8,[MINISTERIO DE DERECHOS SOCIALES Y AGENDA 2030],[Subvenciones],"[Real Decreto 1112/2020, de 15 de diciembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16353],[I. Disposiciones generales]


In [52]:
#for row in tabla_resumen:
#    tabla_resumen.loc[row['Item_id']]['Materias'] = tabla_materias.loc[row['Item_id']]['Materias']

In [53]:
tabla_resumen['Item_id'] = tabla_resumen['Item_id'].astype(str)
tabla_materias['Item_id'] = tabla_materias['Item_id'].astype(str)
tabla_resultados = tabla_resumen.set_index('Item_id').join(tabla_materias.set_index('Item_id'))


In [54]:
tabla_resultados

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Seccion,Alertas,Materias
Item_id,,,,,,,
['BOE-A-2020-16345'],[JEFATURA DEL ESTADO],[Código Penal],"[Ley Orgánica 2/2020, de 16 de diciembre, de m...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Civil, Derecho Penal, Discapacidad]","[Código Penal, Derechos de los ciudadanos, Dis..."
['BOE-A-2020-16346'],[JEFATURA DEL ESTADO],[Agricultura. Alimentanción],"[Ley 8/2020, de 16 de diciembre, por la que se...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Agricultura, Comercio, Derecho Administrativo...","[Agricultura, Andalucía, Ayudas, Comercializac..."
['BOE-A-2020-16347'],[JEFATURA DEL ESTADO],[Contaminación atmosférica],"[Ley 9/2020, de 16 de diciembre, por la que se...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Administrativo, Energía, Industria, M...","[Autorizaciones, Cambios climáticos, Comisione..."
['BOE-A-2020-16348'],"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],[Acuerdo Especial Multilateral RID 8.2020 en v...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Relaciones internacionales, Transportes y trá...","[Acuerdos internacionales, Aparatos y recipien..."
['BOE-A-2020-16349'],[MINISTERIO DEL INTERIOR],[Protección civil],"[Resolución de 16 de diciembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Organización de la Administración, Seguridad ...","[Catástrofes, Cruz Roja Española, Dirección Ge..."
['BOE-A-2020-16350'],"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Energía eléctrica],"[Real Decreto 1106/2020, de 15 de diciembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Consumidores y usuarios, Derecho Mercantil, E...","[Actividades económicas, Ayudas, Certificacion..."
['BOE-A-2020-16351'],"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Subvenciones],"[Real Decreto 1107/2020, de 15 de diciembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],[Asuntos sociales],"[Ayudas, Ayuntamientos, Guerra Civil, Inhumaci..."
['BOE-A-2020-16352'],[MINISTERIO DE POLÍTICA TERRITORIAL Y FUNCIÓN ...,[Organización],"[Real Decreto 1055/2020, de 1 de diciembre, po...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Administración de Justicia, Relaciones intern...","[Administración de Justicia, Bélgica, Cooperac..."
['BOE-A-2020-16353'],[MINISTERIO DE DERECHOS SOCIALES Y AGENDA 2030],[Subvenciones],"[Real Decreto 1112/2020, de 15 de diciembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],[Asuntos sociales],"[Asistencia social, Ayudas, Ayuntamientos, Der..."


In [55]:
#tabla_resultados.to_csv('./BOEs/Resultados.csv', index=False)
tabla_resultados.to_csv('./BOEs/Resultados-BOE-' + hoy + '.csv', index=False)

## Genera Split de ALertas y crea una fila para cada una

### Los items sin Alerta se desechan

In [56]:
tabla_resultados.Alertas.apply(pd.Series) \
    .merge(tabla_resultados, left_index = True, right_index = True) \
    .drop(["Alertas"], axis = 1) \
    .reset_index() \
    .melt(id_vars = ['Item_id','Item_Nombre','Seccion','Departamento','Epigrafe','Item_URL_XML','Materias',], value_name = "Alerta") \
    .drop("variable", axis = 1) \
    .dropna() \
    .sort_values('Alerta')

,Item_id,Item_Nombre,Seccion,Departamento,Epigrafe,Item_URL_XML,Materias,Alerta
7,['BOE-A-2020-16352'],"[Real Decreto 1055/2020, de 1 de diciembre, po...",[I. Disposiciones generales],[MINISTERIO DE POLÍTICA TERRITORIAL Y FUNCIÓN ...,[Organización],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Administración de Justicia, Bélgica, Cooperac...",Administración de Justicia
1,['BOE-A-2020-16346'],"[Ley 8/2020, de 16 de diciembre, por la que se...",[I. Disposiciones generales],[JEFATURA DEL ESTADO],[Agricultura. Alimentanción],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Agricultura, Andalucía, Ayudas, Comercializac...",Agricultura
8,['BOE-A-2020-16353'],"[Real Decreto 1112/2020, de 15 de diciembre, p...",[I. Disposiciones generales],[MINISTERIO DE DERECHOS SOCIALES Y AGENDA 2030],[Subvenciones],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Asistencia social, Ayudas, Ayuntamientos, Der...",Asuntos sociales
6,['BOE-A-2020-16351'],"[Real Decreto 1107/2020, de 15 de diciembre, p...",[I. Disposiciones generales],"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Subvenciones],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Ayudas, Ayuntamientos, Guerra Civil, Inhumaci...",Asuntos sociales
10,['BOE-A-2020-16346'],"[Ley 8/2020, de 16 de diciembre, por la que se...",[I. Disposiciones generales],[JEFATURA DEL ESTADO],[Agricultura. Alimentanción],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Agricultura, Andalucía, Ayudas, Comercializac...",Comercio
5,['BOE-A-2020-16350'],"[Real Decreto 1106/2020, de 15 de diciembre, p...",[I. Disposiciones generales],"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Energía eléctrica],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Actividades económicas, Ayudas, Certificacion...",Consumidores y usuarios
2,['BOE-A-2020-16347'],"[Ley 9/2020, de 16 de diciembre, por la que se...",[I. Disposiciones generales],[JEFATURA DEL ESTADO],[Contaminación atmosférica],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Autorizaciones, Cambios climáticos, Comisione...",Derecho Administrativo
19,['BOE-A-2020-16346'],"[Ley 8/2020, de 16 de diciembre, por la que se...",[I. Disposiciones generales],[JEFATURA DEL ESTADO],[Agricultura. Alimentanción],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Agricultura, Andalucía, Ayudas, Comercializac...",Derecho Administrativo
0,['BOE-A-2020-16345'],"[Ley Orgánica 2/2020, de 16 de diciembre, de m...",[I. Disposiciones generales],[JEFATURA DEL ESTADO],[Código Penal],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Código Penal, Derechos de los ciudadanos, Dis...",Derecho Civil
14,['BOE-A-2020-16350'],"[Real Decreto 1106/2020, de 15 de diciembre, p...",[I. Disposiciones generales],"[MINISTERIO DE LA PRESIDENCIA, RELACIONES CON ...",[Energía eléctrica],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Actividades económicas, Ayudas, Certificacion...",Derecho Mercantil


In [57]:
tabla_resultados['Epigrafe'].str.join(',')

Item_id
['BOE-A-2020-16345']                                Código Penal
['BOE-A-2020-16346']                  Agricultura. Alimentanción
['BOE-A-2020-16347']                   Contaminación atmosférica
['BOE-A-2020-16348']    Acuerdos internacionales administrativos
['BOE-A-2020-16349']                            Protección civil
['BOE-A-2020-16350']                           Energía eléctrica
['BOE-A-2020-16351']                                Subvenciones
['BOE-A-2020-16352']                                Organización
['BOE-A-2020-16353']                                Subvenciones
Name: Epigrafe, dtype: object

In [58]:
from pprint import pprint

## Convierte estructura XML en diccionario

In [59]:
BOE_XML = etree.parse('./BOEs/BOE-A-2020-14457.xml')
raiz=BOE_XML.getroot()

In [60]:
d = [{x.tag: x.text for x in c} for c in raiz]
pprint(d)

[{'departamento': 'Ministerio de Asuntos Exteriores, Unión Europea y '
                  'Cooperación',
  'diario': 'Boletín Oficial del Estado',
  'diario_numero': '304',
  'estado_consolidacion': None,
  'estatus_derogacion': 'N',
  'estatus_legislativo': 'L',
  'fecha_derogacion': None,
  'fecha_disposicion': '20201030',
  'fecha_publicacion': '20201119',
  'fecha_vigencia': '20201101',
  'identificador': 'BOE-A-2020-14457',
  'judicialmente_anulada': 'N',
  'letra_imagen': 'A',
  'numero_oficial': None,
  'origen_legislativo': 'Estatal',
  'pagina_final': '101023',
  'pagina_inicial': '101017',
  'rango': 'Acuerdo Internacional',
  'seccion': '1',
  'subseccion': None,
  'suplemento_letra_imagen': None,
  'suplemento_pagina_final': None,
  'suplemento_pagina_inicial': None,
  'titulo': 'Acuerdo Internacional Administrativo, para el periodo 2020-2021, '
            'entre el Ministerio de Sanidad del Reino de España y la '
            'Organización Mundial de la Salud, para llevar a

In [61]:
pprint(raiz[2].tag)

'texto'


In [62]:
parrafos=raiz.findall("texto/p")
for parrafo in parrafos:
    print(parrafo.text)

ACUERDO INTERNACIONAL ADMINISTRATIVO, PARA EL PERÍODO 2020-2021, ENTRE EL MINISTERIO DE SANIDAD DEL REINO DE ESPAÑA Y LA ORGANIZACIÓN MUNDIAL DE LA SALUD, PARA LLEVAR A CABO ACTIVIDADES DE COOPERACIÓN EN MATERIA DE DONACIÓN Y TRASPLANTE DE ÓRGANOS, CÉLULAS Y TEJIDOS HUMANOS AL AMPARO DEL ACUERDO MARCO DE COOPERACIÓN ENTRE EL REINO DE ESPAÑA Y LA ORGANIZACIÓN MUNDIAL DE LA SALUD, DE 12 DE SEPTIEMBRE DE 2001
Signatarios:
Salvador Illa Roca, Ministro de Sanidad del Reino de España, en virtud del Real Decreto 8/2020, de 12 de enero, por el que se dispone su nombramiento, y en virtud de las facultades otorgadas en el artículo 48.2 de la Ley 40/2015, de 1 de octubre, de Régimen Jurídico del Sector Público.
Mariângela Simão, Subdirectora General de Acceso a Medicamentos, Vacunas y Productos Farmacéuticos de la Organización Mundial de la Salud (en adelante, la OMS), en representación de dicha Organización.
Los signatarios suscriben el presente documento en el marco jurídico del Acuerdo Marco d

# DOGC

In [63]:
URL_HTML_resumen =  "https://dogc.gencat.cat/es/index.html?newLang=es_ES&language=es_ES"

In [77]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get(URL_HTML_resumen)

In [78]:
r.html.links

{'/ca/index.html?newLang=es_ES&language=es_ES',
 '/ca/pdogc_serveis/cerca-republica/',
 '/ca/pdogc_serveis/pdogc_dogc_mobi/',
 '/es/ajuda/mapaweb/',
 '/es/index.html?newLang=es_ES&language=es_ES',
 '/es/pdogc_base_de_dades_del_dogc/',
 '/es/pdogc_canals_interns/pdogc_cercador_de_normativa/?destParam=cercaAdv',
 '/es/pdogc_canals_interns/pdogc_que_es_el_codi_de_verificacio/',
 '/es/pdogc_eadop/',
 '/es/pdogc_enllacos/',
 '/es/pdogc_preguntes_frequents/',
 '/es/pdogc_serveis/',
 '/es/pdogc_serveis/pdogc_normes_urbanistiques_1985-2002/',
 '/es/pdogc_serveis/pdogc_serveis_de_informacio/',
 '/es/pdogc_serveis/pdogc_serveis_de_informacio/pdogc_alertes_dogc/',
 '/es/pdogc_serveis/pdogc_serveis_de_publicacio_al_dogc/Publicacio_Anuncis_situacio_CoV-2/',
 '/es/pdogc_serveis/pdogc_serveis_de_publicacio_al_dogc/pdogc_dogc_eacat/index.html',
 '/es/pdogc_sobre_el_dogc/',
 'http://bop.diba.es/',
 'http://llibreria.gencat.cat/index.php?language=es',
 'http://web.gencat.cat/es/temes/',
 'http://www.boe

In [109]:
r.html.xpath('//*[@id="titol"]/span/text()')

[' DOGC núm.\r\n\t\t\t\t\t 8297 (17/12/2020)']

In [99]:
r.html.xpath('//*[@id="titol"]/h3/text()')


['\r\n\t\t\t\t\tÚltimo DOGC publicado\r\n\t\t\t\t']

In [94]:
r.html.lxml.tag

'html'

In [97]:
r.html.lxml.xpath('//*[@id="titol"]/span')

[<Element span at 0x7ff5180ac2c0>]

In [108]:
#import requests
#from lxml import html

response = requests.get(URL_HTML_resumen)

tree = html.fromstring(response.text)

# fetches all useful elements 
useful = tree.xpath('//*[@id="titol"]/span/text()')

print(useful)


[' DOGC núm.\r\n\t\t\t\t\t 8297 (17/12/2020)']


In [116]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

['/es/pdogc_canals_interns/pdogc_sumari_del_dogc/']

In [126]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@name')

['anexos']

In [127]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@value')

['1']

In [135]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form')

[<Element form at 0x7ff519954310>]

In [140]:
useful = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/input[3]/@value')

print(useful)

['8297']


In [169]:
tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

['/es/pdogc_canals_interns/pdogc_sumari_del_dogc/']

In [184]:
cadena = ''
for input in tree.xpath('//*[@id="sumari"]/ul/li[1]/form'):
    argumentos = input.xpath('./input/@name')
    valores = input.xpath('./input/@value')
    print(argumentos, valores)

for indice in range(len(argumentos)):
    cadena += argumentos[indice] + '=' + valores[indice] + '&'

argumento[1]
cadena[:-1]

['anexos', 'language', 'numDOGC', 'seccion'] ['1', 'es_ES', '8297', '0']


'anexos=1&language=es_ES&numDOGC=8297&seccion=0'

## Recoge Nombre Secciones Sumario

In [205]:
import re
seccion = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/a/text()')
seccion = re.sub('(\\r|\\n|\\t|\ )+', '', seccion[0])
seccion

'DISPOSICIONES'

## Recoge Nombre Secciones Sumario

In [288]:
secciones = tree.xpath('//*[@id="sumari"]/ul/li')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion = seccion.xpath('./form/a/text()')
    seccion = re.sub('(\\r|\\n|\\t)+', '', seccion[0])
    print(seccion[1:-1])
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion[1:-1]}, ignore_index=True)


   DISPOSICIONES 
   CARGOS Y PERSONAL
   ANUNCIOS DE LA GENERALIDAD DE CATALUÑA
   ANUNCIOS DE LA ADMINISTRACIÓN LOCAL
   ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA 


In [290]:
df_secciones_sumarios

,Seccion
0,DISPOSICIONES
1,CARGOS Y PERSONAL
2,ANUNCIOS DE LA GENERALIDAD DE CATALUÑA
3,ANUNCIOS DE LA ADMINISTRACIÓN LOCAL
4,ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA


## Recoge Valores para formar URLs Secciones Sumario

In [291]:
URL_base_sumario = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

df_URL_sumarios = pd.DataFrame()
for seccion in secciones:    
    cadena = ''
    for input in seccion.xpath('./form'):
        argumentos = input.xpath('./input/@name')
        valores = input.xpath('./input/@value')
        #print(argumentos, valores)

    for indice in range(len(argumentos)):
        cadena += argumentos[indice] + '=' + valores[indice] + '&'

    URL_sumario = 'https://dogc.gencat.cat' + str(URL_base_sumario[0]) + '?' + str(cadena[:-1])
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8297&seccion=0
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8297&seccion=1
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8297&seccion=2
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8297&seccion=3
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8297&seccion=4


In [254]:
df_URL_sumarios

,URL_Seccion
0,https://dogc.gencat.cat/es/pdogc_canals_intern...
1,https://dogc.gencat.cat/es/pdogc_canals_intern...
2,https://dogc.gencat.cat/es/pdogc_canals_intern...
3,https://dogc.gencat.cat/es/pdogc_canals_intern...
4,https://dogc.gencat.cat/es/pdogc_canals_intern...


In [292]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [293]:
df_sumarios

,Seccion,URL_Seccion
0,DISPOSICIONES,https://dogc.gencat.cat/es/pdogc_canals_intern...
1,CARGOS Y PERSONAL,https://dogc.gencat.cat/es/pdogc_canals_intern...
2,ANUNCIOS DE LA GENERALIDAD DE CATALUÑA,https://dogc.gencat.cat/es/pdogc_canals_intern...
3,ANUNCIOS DE LA ADMINISTRACIÓN LOCAL,https://dogc.gencat.cat/es/pdogc_canals_intern...
4,ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA,https://dogc.gencat.cat/es/pdogc_canals_intern...


In [294]:
df_sumarios['URL_Seccion'][4]

'https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8297&seccion=4'

## Recoge Items en Seccion Disposiciones

In [405]:
### Recoge Items en Seccion Disposiciones

response = requests.get(df_sumarios['URL_Seccion'][0])
sumario_HTML = html.fromstring(response.text)

#seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div[1]/text()')
seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div/text()')
bloques = sumario_HTML.xpath('//*[@id="disposicions"]')
df_disposiciones = pd.DataFrame()
for bloque in bloques: 
    item_name = bloque.xpath('./div/p/text()')
    item_name = re.sub('(\\r|\\n|\\t)+', '', item_name[0])
    pdf_link = bloque.xpath('./div/div/a[4]/@href')

    df_disposiciones = pd.DataFrame({'Item': item_name, 
                                    'PDF_link': pdf_link})

df_disposiciones['Seccion'] = ''

for row in df_disposiciones.index:
    df_disposiciones.iat[row,2] = seccion[0][:-1] 

df_disposiciones

,Item,PDF_link,Seccion
0,"DECRETO LEY 51/2020, de 15 de diciembre, de mo...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...,DISPOSICIONES
1,"DECRETO LEY 51/2020, de 15 de diciembre, de mo...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...,DISPOSICIONES
2,"DECRETO LEY 51/2020, de 15 de diciembre, de mo...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...,DISPOSICIONES
3,"DECRETO LEY 51/2020, de 15 de diciembre, de mo...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...,DISPOSICIONES
4,"DECRETO LEY 51/2020, de 15 de diciembre, de mo...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...,DISPOSICIONES
5,"DECRETO LEY 51/2020, de 15 de diciembre, de mo...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...,DISPOSICIONES
6,"DECRETO LEY 51/2020, de 15 de diciembre, de mo...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...,DISPOSICIONES
7,"DECRETO LEY 51/2020, de 15 de diciembre, de mo...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...,DISPOSICIONES
8,"DECRETO LEY 51/2020, de 15 de diciembre, de mo...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...,DISPOSICIONES
9,"DECRETO LEY 51/2020, de 15 de diciembre, de mo...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...,DISPOSICIONES
